In [39]:
#import necessary libraries
import numpy as np
from random import random
from random import randint
from random import seed
import pandas as pd
import bisect
from scipy.stats import poisson
from scipy.stats import geom
from scipy.stats import binom

In [40]:

georgia = pd.read_csv("georgia_cases_deaths.csv")
indiana = pd.read_csv("indiana_cases_deaths.csv")
                           
georgia_cases= georgia['new_case'].to_numpy()
georgia_deaths= georgia['new_death'].to_numpy()

indiana_cases= indiana['new_case'].astype(float).to_numpy()
indiana_deaths= indiana['new_death'].to_numpy()
print("######## BEGIN Task B #################")


######## BEGIN Task B #################


In [41]:
#Data for deaths and cases for each of the states for Oct21 and Nov21 and Dec21

# georgia_deaths_oct = georgia_deaths[557:580]   
# georgia_deaths_nov = georgia_deaths[581:609]
# georgia_deaths_dec = georgia_deaths[609:632] 
# georgia_deaths_oct_dec = georgia_deaths[557:632] 

# georgia_cases_oct = georgia_cases[557:580] 
# georgia_cases_nov = georgia_cases[581:609]
# georgia_cases_dec = georgia_cases[609:632]
# georgia_cases_oct_dec = georgia_cases[557:632]

# indiana_deaths_oct = indiana_deaths[570:600]   
# indiana_deaths_nov = indiana_deaths[601:626]
# indiana_deaths_dec = indiana_deaths[627:639] 
indiana_deaths_oct_dec = indiana_deaths[570:639] 

# indiana_cases_oct = indiana_cases[570:600] 
# indiana_cases_nov = indiana_cases[601:626]
# indiana_cases_dec = indiana_cases[627:639]
indiana_cases_oct_dec = indiana_cases[570:639]

In [42]:
def MME_poisson(data):
  # Computing the means of sample1 and sample2
  avg = np.mean(data)
  mu = avg
  print("For poisson")
  print("mu = " + str(mu))
  return mu

def MME_geometric(data):
  avg = np.mean(data)
  avg_1 = 1/avg
  p = avg_1
  print("For geometric")
  print("p = " + str(p))
  return p

def MME_binomial(data):
  avg = np.mean(data)
  Sx = np.var(data, ddof=1)
  n = (avg/(1-(Sx/avg)))
  p = avg/n
  print("For binomial")
  print("p = " + str(p))
  print("n = " + str(n))
  return [n, p]

# print("########Georgia Deaths#################")
# MME_poisson(georgia_deaths_oct_dec)
# MME_geometric(georgia_deaths_oct_dec)
# MME_binomial(georgia_deaths_oct_dec)

# print("##############Gerogia cases Deaths"##############)
# MME_poisson(georgia_cases_oct_dec)
# MME_geometric(georgia_cases_oct_dec)
# MME_binomial(georgia_cases_oct_dec)


In [43]:

def KS_test_1sample(D , D1, distribution, datatype, state):
  print("KS 1 sample test for " + datatype)
  D = D[D!=0]
  D1 = D1[D1!=0]
  min_x = min(D1)
  max_x = max(D1)
  total_elements = len(D1)
  D1.sort()
  mu = None
  p = None
  n = None
  D1dict = dict()
  max_diff = 0
  for ele in D1:
    x = int(ele)
    if x not in D1dict:
      D1dict[x] = 1
    else:
      D1dict[x] += 1
  # print(D1dict)
  prev_ecdf = 0
  curr_ecdf = 0
  curr_sum = 0 
  for key in D1dict:
    curr_sum += D1dict[key]
    curr_ecdf = curr_sum/total_elements
    if distribution == "poisson":
      if mu is None:
        mu = MME_poisson(D)
      cdf = poisson.cdf(key, mu)
    elif distribution == "geometric":
      if p is None:
        p = MME_geometric(D)
      cdf = poisson.cdf(key, p)
    elif distribution == "binomial":
      if n is None or p is None:
        ret = MME_binomial(D)
        n = ret[0]
        p = ret[1]
      cdf = binom.cdf(key, n, p)
    else:
      print("Invalid distribution")
      return None
    left = abs(curr_ecdf - cdf)
    right = abs(prev_ecdf - cdf)
    prev_ecdf = curr_ecdf
    max_diff= max(max_diff, max(left, right))
    # print("ecdf = " + str(curr_ecdf) + ":: cdf = " + str(cdf))
  print("p value = " + str(max_diff))
  if max_diff > 0.05:
    print("Rejecting Null hypothesis i.e :: Distribution of covid " + datatype + " for the state " + state + " is not " + distribution)
  else:
    print("Accepting Null hypothesis i.e :: Distribution of covid " + datatype + " for the state " + state + " is indeed " + distribution)


# KS_test_1sample(georgia_deaths_oct_dec, georgia_deaths_oct_dec, "poisson", "deaths", "georgia")
# KS_test_1sample(georgia_deaths_oct_dec, georgia_deaths_oct_dec, "geometric", "deaths", "georgia")
# KS_test_1sample(georgia_deaths_oct_dec, georgia_deaths_oct_dec, "binomial", "deaths", "georgia")

# KS_test_1sample(georgia_deaths_oct_dec, georgia_cases_oct_dec, "poisson", "cases", "georgia")
# KS_test_1sample(georgia_deaths_oct_dec, georgia_cases_oct_dec, "geometric", "cases", "georgia")
# KS_test_1sample(georgia_deaths_oct_dec, georgia_cases_oct_dec, "binomial", "cases", "georgia")

# KS_test_1sample(indiana_deaths_oct_dec, indiana_deaths_oct_dec, "poisson", "deaths", "indiana")
# KS_test_1sample(indiana_deaths_oct_dec, indiana_deaths_oct_dec, "geometric", "deaths", "indiana")
# KS_test_1sample(indiana_deaths_oct_dec, indiana_deaths_oct_dec, "binomial", "deaths", "indiana")

# KS_test_1sample(indiana_deaths_oct_dec, indiana_cases_oct_dec, "poisson", "cases", "indiana")
# KS_test_1sample(indiana_deaths_oct_dec, indiana_cases_oct_dec, "geometric", "cases", "indiana")
# KS_test_1sample(indiana_deaths_oct_dec, indiana_cases_oct_dec, "binomial", "cases", "indiana")

KS_test_1sample(georgia_deaths_oct_dec, indiana_deaths_oct_dec, "poisson", "deaths", "indiana")
KS_test_1sample(georgia_deaths_oct_dec, indiana_deaths_oct_dec, "geometric", "deaths", "indiana")
KS_test_1sample(georgia_deaths_oct_dec, indiana_deaths_oct_dec, "binomial", "deaths", "indiana")

KS_test_1sample(georgia_deaths_oct_dec, indiana_cases_oct_dec, "poisson", "cases", "indiana")
KS_test_1sample(georgia_deaths_oct_dec, indiana_cases_oct_dec, "geometric", "cases", "indiana")
KS_test_1sample(georgia_deaths_oct_dec, indiana_cases_oct_dec, "binomial", "cases", "indiana")
  


KS 1 sample test for deaths
For poisson
mu = 68.57692307692308
p value = 0.6184007206616742
Rejecting Null hypothesis i.e :: Distribution of covid deaths for the state indiana is not poisson
KS 1 sample test for deaths
For geometric
p = 0.014582164890633763
p value = 1.0
Rejecting Null hypothesis i.e :: Distribution of covid deaths for the state indiana is not geometric
KS 1 sample test for deaths
For binomial
p = -16.67721289300914
n = -4.112013411165938
p value = 1.0
Rejecting Null hypothesis i.e :: Distribution of covid deaths for the state indiana is not binomial
KS 1 sample test for cases
For poisson
mu = 68.57692307692308
p value = 1.0
Rejecting Null hypothesis i.e :: Distribution of covid cases for the state indiana is not poisson
KS 1 sample test for cases
For geometric
p = 0.014582164890633763
p value = 1.0
Rejecting Null hypothesis i.e :: Distribution of covid cases for the state indiana is not geometric
KS 1 sample test for cases
For binomial
p = -16.67721289300914
n = -4.11

In [44]:
def KS_test_2sample(D1 , D2, datatype):
  print("KS 2 sample test for " + datatype)
  min_x_D1 = min(D1)
  max_x_D1 = max(D1)
  total_elements_D1 = len(D1)
  D1.sort()
  D1dict = dict()
  max_diff = 0
  for ele in D1:
    x = int(ele)
    if x not in D1dict:
      D1dict[x] = 1
    else:
      D1dict[x] += 1
  # print(D1dict)
  min_x_D2 = min(D2)
  max_x_D2 = max(D2)
  total_elements_D2 = len(D2)
  D2.sort()
  D2dict = dict()
  for ele in D2:
    x = int(ele)
    if x not in D2dict:
      D2dict[x] = 1
    else:
      D2dict[x] += 1
  # print(D2dict)
  curr_sum = 0
  for key in D1dict:
    curr_sum += D1dict[key]
    D1dict[key] = curr_sum/total_elements_D1
  
  curr_sum = 0
  for key in D2dict:
    curr_sum += D2dict[key]
    D2dict[key] = curr_sum/total_elements_D2
  
  list1 = list(D1dict.keys())
  rev_list1 = list1[::-1]
  list2 = list(D2dict.keys())
  rev_list2 = list2[::-1]
  # print(list1)
  # print(list2)
  for key1 in D1dict:
    left_key2 = bisect.bisect_left(list2, key1)
    right_key2 = bisect.bisect_left(list2, key1)
    left_key1 = bisect.bisect_left(list1, key1)
    right_key1 = bisect.bisect_left(list1, key1)
    left = abs(D2dict[list2[left_key2 - 1]] - D1dict[list1[left_key1 - 1]])
    right = abs(D2dict[list2[right_key2]] - D1dict[list1[right_key1]])
    max_diff = max(max_diff, max(left, right))

  print("p value = " + str(max_diff))
  if max_diff > 0.05:
    print("Rejecting Null hypothesis i.e :: Distribution of covid " + datatype + " is not same for both states")
  else:
    print("Accepting Null hypothesis i.e :: Distribution of covid " + datatype + "is indeed same for both states")

KS_test_2sample(indiana_cases_oct_dec , georgia_cases_oct_dec, "cases")
KS_test_2sample(indiana_deaths_oct_dec , georgia_deaths_oct_dec, "deaths")

KS 2 sample test for cases
p value = 0.35768115942028983
Rejecting Null hypothesis i.e :: Distribution of covid cases is not same for both states
KS 2 sample test for deaths
p value = 0.24637681159420288
Rejecting Null hypothesis i.e :: Distribution of covid deaths is not same for both states


In [45]:
#Permustation test
def generate_perm(D1, D2):
  len1 = len(D1)
  len2 = len(D2)

  D = D1
  D = np.append(D1, D2)
  # print(D)

  length = len(D)
  last = length - 1
  for i in range(length):
    rand = randint(0, length - i - 1)
    # print(rand)
    D[[rand, length - i - 1]] = D[[length - i - 1, rand]]
  # print(D)
  D = np.split(D, 2)
  D1 = D[0]
  D2 = D[1]
  return [D1, D2]
  

def perm_test(D1, D2, datatype):
  print("Permutation test for " + datatype)
  len1 = len(D1)
  len2 = len(D2)
  og_mean1 = np.mean(D1)
  og_mean2 = np.mean(D2)
  og_mean_diff = abs(og_mean1 - og_mean2)
  perm_D1 = []
  perm_D2 = []
  count = 0
  for i in range(1000):
    D = generate_perm(D1, D2)
    perm_D1 = D[0]
    perm_D2 = D[1]
    # print(perm_D1)
    # print(perm_D2)
    mean1 = np.mean(perm_D1)
    mean2 = np.mean(perm_D2)
    mean_diff = abs(mean1 - mean2)
    if mean_diff > og_mean_diff:
      count+=1
  
  p_value = (count/1000)
  print("p_value = " + str(p_value))
  if p_value < 0.05:
    print("Rejeting Null Hypothesis i.e Distribution of " + datatype + " of two states is not same")
  else:
    print("Failed to Reject Null Hypothesis i.e Distribution of " + datatype + " of two states is indeed same")
  

perm_test(indiana_deaths_oct_dec, georgia_deaths_oct_dec, "deaths")
perm_test(indiana_cases_oct_dec, georgia_cases_oct_dec, "cases")





Permutation test for deaths
p_value = 0.003
Rejeting Null Hypothesis i.e Distribution of deaths of two states is not same
Permutation test for cases
p_value = 0.001
Rejeting Null Hypothesis i.e Distribution of cases of two states is not same
